# 관심있는 데이터소스로 RAG

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# OpenAI 임베딩 모델
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# VectorStore 에 임베딩 후 저장(In memory)
vectorstore = FAISS.from_documents(documents, embedding=embedding)

In [4]:
# 1. Load
import glob
from langchain_community.document_loaders import PyMuPDFLoader

file_list = glob.glob("./mystudy/*.pdf")  # mystudy 폴더 내 모든 pdf 파일 경로 리스트
all_docs = []

for file_path in file_list:
    loader = PyMuPDFLoader(file_path)
    docs = loader.load()
    all_docs.extend(docs)

print("총 로드한 페이지 수:", len(all_docs))

총 로드한 페이지 수: 960


In [5]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 500글자당 1 청크 / 50글자는 겹치게 나눈다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(all_docs)

print("분할 후 청크 수", len(split_docs))

분할 후 청크 수 4235


In [6]:
# 3. 임베딩, 4. 벡터스토어 저장
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_docs, embedding=embedding)

# Test
vectorstore.similarity_search("에이전트", k=4)

[Document(id='669bfc32-b293-4480-bba5-74829ffb6e89', metadata={'producer': 'Skia/PDF m127', 'creator': 'Chromium', 'creationdate': '2025-06-09T05:34:42+00:00', 'source': './mystudy\\멀티에이전트 시스템 취업 준비 완벽 가이드.pdf', 'file_path': './mystudy\\멀티에이전트 시스템 취업 준비 완벽 가이드.pdf', 'total_pages': 4, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-06-09T05:34:42+00:00', 'trapped': '', 'modDate': "D:20250609053442+00'00'", 'creationDate': "D:20250609053442+00'00'", 'page': 1}, page_content='수 있게 하며, 감독자 에이전트가 이들을 연결하는 역할을 합니다\n.\nAWS에서 제안하는 도메인 인식 데이터 전처리 파이프라인은 실무에서 활용할 수 있는 고급 멀티에이\n전트 협업 접근법의 좋은 예시입니다\n. 이 파이프라인은 분류, 변환, 메타데이터 추출, 도메인별 작업\n을 처리하는 여러 전문화된 에이전트로 구성되어 있습니다\n. 각 에이전트는 특정 기능을 담당하며,\n이들을 조율함으로써 다양한 형태의 비구조적 데이터 수집과 변환을 자동화할 수 있습니다\n.\n2단계: MCP와 멀티모달 기술 심화 (6-8주)\n[6]\n[6]\n[6]\n[7]\n[7]\n[7]\n[8]\n[8]\n[9]\n[9]\n3단계: 고급 멀티에이전트 아키텍처 구현 (8-10주)\n[10]\n[10]\n[10]\n[11]\n[11]\n[11]\n실무 적용 프로젝트 포트폴리오 구성'),
 Document(id='30fdd759-bd6b-4895-a130-8d8

In [7]:
# 5. RAG
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain import hub
from pprint import pprint

# Prompt 세팅
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""당신은 제공된 PDF 자료를 기반으로 답변하는 전문 어시스턴트입니다.
    
주어진 문서 내용을 바탕으로 질문에 정확하고 상세하게 답변해주세요.
문서에 정보가 없다면 "제공된 자료에서 해당 정보를 찾을 수 없습니다"라고 답하세요.

문서 내용:
{context}

질문: {question}

답변:"""
)

# LLM 모델
llm = ChatOpenAI(model="gpt-5-nano", temperature=0.3)

# 검색기 생성(retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

pprint(chain.invoke("이 사람이 공부하고 배운 내용중 가치있는 부분이나 임플리케이션이 있다면 정리하고 요약해줘"))

('다음은 주어진 문서들을 바탕으로 이 사람이 공부하고 배운 내용의 가치 있는 부분과 시사점(임플리케이션)을 정리한 요약입니다.\n'
 '\n'
 '1) 관계와 심리 인사이트(선물/호의의 사례 인식)\n'
 '- 핵심 내용\n'
 '  - 선물이나 호의에 대해 바로 보답해야 한다는 의무감이 부담으로 작용하는 경우가 많다.\n'
 '  - 받는 것의 기쁨을 상대도 느낄 수 있도록 마음가짐을 바꿔보면 관계의 긴장을 완화할 수 있다.\n'
 '  - 모든 호의에 즉시 보답하지 않아도 되고, 나중에 내가 할 수 있을 때 내 방식으로 돌려주면 된다는 생각이 관계의 균형을 자연스럽게 '
 '회복시키는 데 도움이 된다.\n'
 '- 임플리케이션\n'
 '  - 대인관계에서 불필요한 긴장을 줄이고 상호 배려를 지속적으로 유지하는 데 유용한 사고방식이다.\n'
 '  - 팀이나 협업 상황에서 기여와 보답의 흐름을 장기적으로 바라볼 수 있어 스트레스를 낮추고 신뢰를 구축하는 데 도움이 된다.\n'
 '  - “상대의 기쁜 마음”에 집중하는 태도는 협업과 피드백 문화 형성에 긍정적이다.\n'
 '\n'
 '2) 문제 해결 학습 방법론(Decomposition/단순화 매뉴얼)\n'
 '- 핵심 내용\n'
 '  - 6-step 루틴으로 문제를 체계적으로 다룬다: Clarify, Examples, Constraints & Brute-Force, '
 'Decompose & Plan, Code in(추정상 5단계까지 제시).\n'
 '  - 각 단계의 핵심 활동\n'
 '    - Clarify: 입력/출력/제약/엣지 케이스를 재진술하고 주석으로 옮겨 오독을 줄임.\n'
 '    - Examples: 정상/엣지/반례의 테스트를 직접 만들어 조건을 밝힘.\n'
 '    - Constraints & Brute-Force: 문제의 한계와 직관적(느린) 해법을 먼저 확인해 두면 이후 최적화가 쉬움.\n'
 '    - Decompose & Plan: 문제를 MECE 원칙으로 분할하고 의사